# Storing, retreiving, and processing single cell RNA-seq data with TileDB

In this notebook we will attempt to store single cell data in TileDB. We will use 3k PBMCs scRNA-seq data from 10x. Code for downloading and loading data into Anndata format is adapted from https://scanpy-tutorials.readthedocs.io/en/latest/pbmc3k.html .

In [1]:
from pathlib import Path
import tempfile

import numpy as np
import pandas as pd
import scanpy as sc
import tiledb

Now we can get the data. Uncomment the two lines in the cell below to download the data.

In [2]:
#!wget http://cf.10xgenomics.com/samples/cell-exp/1.1.0/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz
#!tar -xzf pbmc3k_filtered_gene_bc_matrices.tar.gz

In [2]:
adata = sc.read_10x_mtx(
    './filtered_gene_bc_matrices/hg19/',  # the directory with the `.mtx` file
    var_names='gene_symbols',                  # use gene symbols for the variable names (variables-axis index)
)    

Here's a peek into the `AnnData` object. Some of the attributes are for convenience, some of them are specific to AnnData internals, and others actually hold the data. `AnnData` documentation is here: https://icb-anndata.readthedocs-hosted.com/en/stable/anndata.AnnData.html

Here are the more important ones. `adata.X` contains the expression values, and has cells (obs) rows by genes (var) columns. We can obtain a dense numpy array representation of the sparse data by calling `adata.X.toarray()`

In [3]:
adata.X

<2700x32738 sparse matrix of type '<class 'numpy.float32'>'
	with 2286884 stored elements in Compressed Sparse Row format>

Note that `adata.X` is a very sparse array.

In [4]:
print(f"First row is {1 - (np.count_nonzero(adata.X[0].toarray()) / adata.X[0].shape[1]):.2%} empty")
adata.X[0].toarray()

First row is 97.61% empty


array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

 `adata.obs` contains the cell barcodes, same number of barcodes as rows of `adata.X`. These are in a 0-column `pandas` `DataFrame`, of which the barcodes form the index. This is presumably done so that things like cluster labels can be added in after analysis. This index can be conveniently accessed with `adata.obs_names`

In [5]:
adata.obs

""
AAACATACAACCAC-1
AAACATTGAGCTAC-1
AAACATTGATCAGC-1
AAACCGTGCTTCCG-1
AAACCGTGTATGCG-1
...
TTTCGAACTCTCAT-1
TTTCTACTGAGGCA-1
TTTCTACTTCCTCG-1
TTTGCATGAGAGGC-1


`adata.var` contains the gene names and Ensembl IDs, same as the number of columns of `adata.X`. In `AnnData` these are stored in a 1-column `pandas` `DataFrame`. The gene name is used as the `DataFrame` index and the Ensembl IDs are the values.

In [6]:
adata.var

,gene_ids
MIR1302-10,ENSG00000243485
FAM138A,ENSG00000237613
OR4F5,ENSG00000186092
RP11-34P13.7,ENSG00000238009
RP11-34P13.8,ENSG00000239945
...,...
AC145205.1,ENSG00000215635
BAGE5,ENSG00000268590
CU459201.1,ENSG00000251180
AC002321.2,ENSG00000215616


Now we have some idea of what we need to store in TileDB. We want to store the expression values in a cell by gene dense matrix. We need at the minimum two pieces of metadata for each expression value: the gene name or ID, and the cell barcode. Even though the matrix is mathematically sparse, if we used a `tiledb.SparseMatrix` it would be more difficult materialize gene and cell information for missing values.

The tile extents are "optimized" for random access of a subset of genes across all cells. Note that the tile extent of the cell dimension spans all of the cells, while the tile extent of the gene dimension is very small, so we can pluck out data for individual genes fairly efficiently.

We pull the array dimensions from the `AnnData` object. In TileDB terminology domain is inclusive of endpoints. We swap out the attribute dtype for the counts from `adata.X` from `np.float32` to `np.uint16`. Using a float doesn't make sense for the counts, which are strictly non-negative and integer valued. In order to write the data, we need gather all of the attributes together, i.e. for each count value we want to write, we need to gather the appropriate gene name, gene ID, and barcode.

We must specify the attributes in the dict in the same order as they are defined in the schema, this seems to be a bug in the TileDB Python API: https://github.com/TileDB-Inc/TileDB-Py/issues/299

In [4]:
array_name = "3k_pbmc_array"
array_path = Path(array_name)
if not array_path.exists():
    tiledb.DenseArray.create(array_name, schema)
    dom = tiledb.Domain(
        tiledb.Dim(name="cells", domain=(0, adata.X.shape[0] - 1), tile=adata.X.shape[0], dtype=np.uint32),
        tiledb.Dim(name="genes", domain=(0, adata.X.shape[1] - 1), tile=2, dtype=np.uint32),
    )
    schema = tiledb.ArraySchema(
        domain=dom,
        sparse=False,
        attrs=(
            tiledb.Attr(name="counts", dtype=np.uint16, var=False),
            tiledb.Attr(name="cell_barcode", dtype=np.bytes_, var=True),
            tiledb.Attr(name="gene_name", dtype=np.bytes_, var=True),
            tiledb.Attr(name="gene_ensembl_id", dtype=np.bytes_, var=True)
        )
    )
    with tiledb.DenseArray(array_name, mode='w') as A:
        to_write = {
            "counts": adata.X.toarray().astype(np.uint16),
            "cell_barcode": np.tile(np.array(adata.obs_names, dtype=bytes), (adata.X.shape[1], 1)).T,
            "gene_name": np.tile(np.array(adata.var_names.values, dtype=bytes), (adata.X.shape[0],1)),
            "gene_ensembl_id": np.tile(np.array(adata.var.values.T, dtype=bytes), (adata.X.shape[0],1)),
        }
        A[:,:] = to_write

Now that's we've created the array, we can subset it relatively quickly by gene. Remember that each column of the array represents one gene.

In [5]:
with tiledb.DenseArray(array_name, mode='r') as A:
    print(A[:,0])

OrderedDict([('counts', array([0, 0, 0, ..., 0, 0, 0], dtype=uint16)), ('cell_barcode', array([b'AAACATACAACCAC-1', b'AAACATTGAGCTAC-1', b'AAACATTGATCAGC-1', ...,
       b'TTTCTACTTCCTCG-1', b'TTTGCATGAGAGGC-1', b'TTTGCATGCCTCAC-1'],
      dtype=object)), ('gene_name', array([b'MIR1302-10', b'MIR1302-10', b'MIR1302-10', ..., b'MIR1302-10',
       b'MIR1302-10', b'MIR1302-10'], dtype=object)), ('gene_ensembl_id', array([b'ENSG00000243485', b'ENSG00000243485', b'ENSG00000243485', ...,
       b'ENSG00000243485', b'ENSG00000243485', b'ENSG00000243485'],
      dtype=object))])


## Another approach

Dumping everything into one dense array isn't the only way to do things. Perhaps more in line with the `AnnData` approach would be to store the core expression matrix `adata.X` as a *sparse* array, and store the row and column labels separately as dense vectors. We can accomplish this using TileDB groups. Since we don't want to write any data where the expression values are zero, we need to find the indices of the non-zero coordinates and their corresponding values.

In [112]:
group_name = "3k_pbmc_group"
group_path = Path(group_name)
if not group_path.exists():
    tiledb.group_create(group_name)
counts_array_path = group_path / "3k_pbmc_counts"
if not counts_array_path.exists():
    dom = tiledb.Domain(
        tiledb.Dim(name="cells", domain=(0, adata.X.shape[0] - 1), tile=adata.X.shape[0], dtype=np.uint32),
        tiledb.Dim(name="genes", domain=(0, adata.X.shape[1] - 1), tile=2, dtype=np.uint32),
    )
    schema = tiledb.ArraySchema(
        domain=dom,
        sparse=True,
        attrs=(tiledb.Attr(name="counts", dtype=np.uint16,),)
    )
    tiledb.SparseArray.create(str(counts_array_path), schema)
    with tiledb.SparseArray(str(counts_array_path), mode='w') as A:
        x, y = np.nonzero(adata.X)
        A[x, y] = {"counts": adata.X.toarray()[np.nonzero(adata.X)].astype(np.uint16)}

Let's also add the barcode and gene metadata to the group. The gene names will have two attributes, one for the gene names and another for the Ensembl IDs. Note that we can drop the `np.tile` since we want to write just a single vector.

In [67]:
gene_array_name = "genes"
dom = tiledb.Domain(
    tiledb.Dim(name="genes", domain=(0, adata.X.shape[1] - 1), tile=adata.X.shape[1], dtype=np.uint32),
)
schema = tiledb.ArraySchema(
    domain=dom,
    sparse=False,
    attrs=(
        tiledb.Attr(name="gene_name", dtype=np.bytes_, var=True),
        tiledb.Attr(name="gene_ensembl_id", dtype=np.bytes_, var=True)
    )
)
gene_array_path = group_path / gene_array_name
if not gene_array_path.exists():
    tiledb.DenseArray.create(str(gene_array_path), schema)
with tiledb.DenseArray(str(gene_array_path), mode='w') as A:
    data = {
        "gene_name": np.array(adata.var_names, dtype=bytes),
        "gene_ensembl_id": np.array(adata.var.values.flatten(), dtype=bytes),
    }
    A[:] = data

We can do the same with the cell barcodes.

In [68]:
barcode_array_name = "cell_barcodes"
barcode_array_path = group_path / barcode_array_name
if not barcode_array_path.exists():
    dom = tiledb.Domain(
        tiledb.Dim(name="cell_barcodes", domain=(0, adata.X.shape[0] - 1), tile=adata.X.shape[0], dtype=np.uint32),
    )
    schema = tiledb.ArraySchema(
        domain=dom,
        sparse=False,
        attrs=(
            tiledb.Attr(name="barcodes", dtype=np.bytes_, var=True),
        )
    )
    tiledb.DenseArray.create(str(barcode_array_path), schema)
    with tiledb.DenseArray(str(barcode_array_path), mode='w') as A:
        A[:] = {
            "barcodes": np.array(adata.obs_names, dtype=bytes)
        }

Now we can see all of the arrays in our group using the TileDB API.

In [69]:
tiledb.ls(group_name, lambda obj_path, obj_type: print("URI:", obj_path, ", object type", obj_type))

URI: file:///Users/paul/notebooks/tiledb-experiments/notebooks/3k_pbmc_group/3k_pbmc_counts , object type array
URI: file:///Users/paul/notebooks/tiledb-experiments/notebooks/3k_pbmc_group/cell_barcodes , object type array
URI: file:///Users/paul/notebooks/tiledb-experiments/notebooks/3k_pbmc_group/genes , object type array


The underlying data is still the same as before. However, now the cell and gene attributes aren't stored with the counts, just the expressions. Also note that when we get the data now, only the nonzero expressions are returned.

In [119]:
with tiledb.SparseArray(str(counts_array_path), "r") as A:
    print(A[0,:]["counts"])

[ 1  1  2  1  1  1  1 41  1  1  1  1  2  6  1  1  1  1  1  1  2  1 15  1
  1  1  1  1  1 11  1  1  5  1  1  1  1  1  1  1  1  1  1  1  1  3  2  1
 15  2  1  1  1  1  1  4  1  1  1  1  1  1  1  2  1  1  1  1  1  1  1 13
  1  1  1  1  2  1  5  2  2 22  1  1  2  1  1  1  2  6  1  1  1  2  1  1
  1  1 11  1  1  1  2  1  1  1  1  1  1  1  1  3  1  2  1  2  1  4  1  1
  1  1 11  1  1  1  1  1  1  1  1  1  2 17  1  8  1 17 11  1  1  1  1  1
 14  1  2  1  1  1  1  5  1  1  1  1  1  1  2  1  2  1  3  1  1  7  1  2
  1  1 23  1  1  1  1  1  2  1  2  1  1  1  2  1 13  3  1  2 19  1  1  1
  3  2  1  5  1  1  1  1  4  7  1  1  1  1  1  2  2  1  1  1  1  1  1 18
  1  1 15  1  1  5  2  2  1  2  1  1  8  1  1  1  1  1 21  5  5  8  5  1
  5  2  1  3  1  1  1 22  2  1 10  1  1 10  1  1  1  1  1 19  1  1  3  1
  1  2  1  1  1  1  1 37  1  1  1  1  1  1  1  1  1  1  1 17  1  1  1  2
  1  2  3  2  1  1  1  1  1  1  1  1  2  1  1  1  1  1  1  1  1  1  2  1
 12  1  1 47  1  1  1  1  1  1  1  2  1  1  1 27  1